In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

%matplotlib inline

In [35]:
# Heart Attack Data
ha_costs_df = pd.read_csv('../data/mmd_heart_attack_data.csv')
ha_costs_df = ha_costs_df.query('state == "TENNESSEE"')
tn_ha_costs = ha_costs_df[['county','urban','analysis_value']]

# Cancer Data
cancer_costs_df = pd.read_csv('../data/mmd_cancer_data.csv')
cancer_costs_df = cancer_costs_df.query('state == "TENNESSEE"')
tn_cancer_costs = cancer_costs_df[['county','urban','analysis_value']]

# Income data
income_df = pd.read_csv('../data/irs_county_2016.csv')
income_df = income_df.query('STATE == "TN"')
column_list = ['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']
income_df = income_df[column_list]
new_column_list = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']
column_name_dict = dict(zip(column_list,new_column_list))
income_df = income_df.rename(columns = column_name_dict)
value_key = income_df.income_bucket.unique().tolist()
value_list = ['Total', 'Under $1', 'Between 1 and $10,000', 'Between 10,000 and $25,000','Between 25,000 and $50,000', 'Between 50,000 and $75,000', 'Between 75,000 and $100,000', 'Between 100,000 and $200,000', '$200,000 or more'
        ]
income_dict = dict(zip(value_key,value_list[1:]))
income_df.income_bucket = income_df.income_bucket.replace(income_dict)
income_county_agg = income_df.groupby(['county']).sum()
income_county_agg = income_county_agg.reset_index()
income_county_agg['avg_income'] = round(income_county_agg.total_inc_amt * 1000 / income_county_agg.return_count)

tn_ha_cost2 = pd.merge(income_county_agg[['county','avg_income']], tn_ha_costs, on="county", how = 'left')
tn_ha_cost2['cost_income_ratio'] = tn_ha_cost2.analysis_value / tn_ha_cost2.avg_income

tn_cancer_cost2 = pd.merge(income_county_agg[['county','avg_income']], tn_cancer_costs, on="county", how = 'left')
tn_cancer_cost2['cost_income_ratio'] = tn_cancer_cost2.analysis_value / tn_cancer_cost2.avg_income

In [36]:
# other downloaded data
population = pd.read_csv('../Data/PEP_2016_PEPANNRES_with_ann.csv',header=[0,1])
population = population.iloc[1:,[2,-1]]
population.columns = ['county','total_population']
population_change = pd.read_csv('../Data/PEP_2016_PEPTCOMP_with_ann.csv',header=[0,1])
population_change = population_change.iloc[1:,[2,4,7]]
population_change.columns = ['county','natural_change','migration_change']
age_sex = pd.read_csv('../Data/PEP_2016_PEPAGESEX_with_ann.csv',skiprows = 0, header=1)
age_sex = age_sex[['Geography','Population Estimate (as of July 1) - 2016 - Male; Total',
                   'Population Estimate (as of July 1) - 2016 - Both Sexes; 65 years and over',
                  'Population Estimate (as of July 1) - 2016 - Male; 65 years and over']]
age_sex.columns = ['county','male_population','male_65_and_over','female_65_and_over']
other_data = pd.merge(population,population_change,on='county',how='outer')
other_data = pd.merge(other_data, age_sex, on='county', how='outer')
other_data.county = other_data.county.str[0:-11]

In [37]:
all_data = pd.merge(tn_cancer_cost2,other_data,on='county',how='right')
all_data = pd.merge(all_data, tn_ha_cost2,on='county',how='left',suffixes=('_cancer', '_ha'))

In [38]:
len(all_data.county.values)

95

In [39]:
all_data.head()

,county,avg_income_cancer,urban_cancer,analysis_value_cancer,cost_income_ratio_cancer,total_population,natural_change,migration_change,male_population,male_65_and_over,female_65_and_over,avg_income_ha,urban_ha,analysis_value_ha,cost_income_ratio_ha
0,Anderson County,53382.0,Urban,15454.0,0.289498,75936,-689,1580,36905,14972,6590,53382.0,Urban,42749.0,0.800813
1,Bedford County,47128.0,Rural,19219.0,0.407804,47484,1056,1323,23343,7162,3205,47128.0,Rural,43661.0,0.926434
2,Benton County,41297.0,Rural,21315.0,0.516139,16014,-511,34,7896,3730,1754,41297.0,Rural,44423.0,1.075696
3,Bledsoe County,39744.0,Rural,19250.0,0.484350,14675,-53,1723,8574,2669,1333,39744.0,Rural,41240.0,1.037641
4,Blount County,56786.0,Urban,17073.0,0.300655,128670,-90,5361,62432,24938,11218,56786.0,Urban,38968.0,0.686225
